In [28]:
import foolbox
import torch
import torchvision.models as models
import numpy as np
from PIL import Image

# instantiate the model
resnet18 = models.resnet18(pretrained=True).eval()
mean = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
std = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))
fmodel = foolbox.models.PyTorchModel(
    resnet18, bounds=(0, 1), num_classes=1000, preprocessing=(mean, std))

# get source image and label
image, label = foolbox.utils.imagenet_example(data_format='channels_first')
print (image.shape)
print(type(image))
ori_image = Image.fromarray(np.uint8(image.swapaxes(0,2).swapaxes(0,1)))
ori_image.show()
image = image / 255.  # because our model expects values in [0, 1]

print('label', label)
print('predicted class', np.argmax(fmodel.predictions(image)))

# apply attack on source image
attack = foolbox.attacks.FGSM(fmodel)
adversarial = attack(image, label)

print('adversarial class', np.argmax(fmodel.predictions(adversarial)))
Image.fromarray(np.uint8(adversarial.swapaxes(0,2).swapaxes(0,1) * 255)).show()
diff = np.abs(image-adversarial)
diff = diff / np.max(diff)
Image.fromarray(np.uint8(diff.swapaxes(0,2).swapaxes(0,1) * 255)).show()

(3, 224, 224)
<class 'numpy.ndarray'>
label 282
predicted class 282
adversarial class 281
